<b> Описание задачи</b>

Задача проекта — проанализировать базу данных сервиса книг по подписке.

В базе данных — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.  

База данных состоит из следующих таблиц:  
- Таблица books - содержит данные о книгах  
- Таблица authors - содержит данные об авторах  
- Таблица publishers - содержит данные об издательствах  
- Таблица ratings - содержит данные о пользовательских оценках книг  
- Таблица reviews - содержит данные о пользовательских обзорах

В ходе выполнения проекта предстоит выполнить следующие задания:  
- Посчитать, сколько книг вышло после 1 января 2000 года;  
- Для каждой книги посчитать количество обзоров и среднюю оценку;  
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг для книг с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

По факту решения каждого задания будут сформулированы выводы.

In [85]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [86]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [87]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [88]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [89]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [90]:
#Посчитать, сколько книг вышло после 1 января 2000 года;
query= '''SELECT count(book_id)
            FROM books
            where publication_date> '2000-01-01'  '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


819 книг из базы данных books удовлетворяют условию выпуска позднее 01.01.2000.

In [91]:
#проверка суммы ревью в таблице reviews
query = '''SELECT count(review_id) count_reviews FROM reviews '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count_reviews
0,2793


In [92]:
# Для каждой книги посчитать количество обзоров и среднюю оценку;
# ПОСЛЕ ИСПРАВЛЕНИЯ
query= '''
with 
rev_grouped as (

select book_id,
       count(review_id) review_counts
       from reviews 
       group by book_id)
       
select b.book_id,
       b.title,
       rev_grouped.review_counts review_counts,
       avg(rat.rating) average_rating
from books b
join rev_grouped on b.book_id=rev_grouped.book_id
join ratings rat on b.book_id=rat.book_id
group by b.book_id, rev_grouped.review_counts
limit 10'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,review_counts,average_rating
0,716,The Epic of Gilgamesh,2,3.000000
1,499,Pandora's Star,2,3.000000
2,838,The Princess Diaries (The Princess Diaries #1),3,3.875000
3,796,The Midwife's Apprentice,2,4.000000
4,455,Murder at the Vicarage (Miss Marple #1),3,4.400000
5,522,Practical Demonkeeping (Pine Cove #1),2,4.500000
6,422,Matilda,4,4.421053
7,476,Nicolae (Left Behind #3),2,4.000000
8,843,The Quiet American,2,4.500000
9,958,Veronika Decides to Die,4,3.400000


Для каждой книги выполнен расчет количества обзоров и среднего рейтинга.

In [93]:
# проверка суммы ревью при составлении запроса после корректировки
query= '''
with 
rev_grouped as (

select book_id,
       count(review_id) review_counts
       from reviews 
       group by book_id),

rev_count as (
select b.book_id,
       b.title,
       rev_grouped.review_counts review_counts,
       avg(rat.rating) average_rating
from books b
join rev_grouped on b.book_id=rev_grouped.book_id
join ratings rat on b.book_id=rat.book_id
group by b.book_id, rev_grouped.review_counts )

select sum(review_counts)
from rev_count
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,sum
0,2793.0


In [94]:
#Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
query= '''
select  pub.publisher_id,
        pub.publisher,
        count(b.book_id)
from publishers pub
join books b on pub.publisher_id=b.publisher_id
where b.num_pages>50
group by pub.publisher_id
order by count(b.book_id) desc
limit 1'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,count
0,212,Penguin Books,42


Издательство Penguin Books	выпустило 42 книги толще 50 страниц - это максимальный показатель.

In [95]:
#Определить автора с самой высокой средней оценкой книг для книг с 50 и более оценками;
query = '''
select a.author_id,
       a.author,
       count(rat.rating_id) count_of_ratings,
       AVG(rat.rating) average_rating
from authors a 
join books b on a.author_id=b.author_id
join ratings rat on b.book_id=rat.book_id

group by a.author_id
having count(rat.rating_id)>50
order by AVG(rat.rating) desc
limit 1'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,count_of_ratings,average_rating
0,236,J.K. Rowling/Mary GrandPré,312,4.288462


Самая высокая средняя оценка книг для книг с 50 и более оценками принадлежит J.K. Rowling/Mary GrandPré	.

In [96]:
#Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
query = ''' 

with rat_48 as (
select rat.username user_name
from ratings rat 
group by rat.username
having count(rat.rating_id)>48),

rev_count as (
select count(rev.review_id) total_count
from reviews  rev
join rat_48 on rev.username=rat_48.user_name
group by rev.username)

select avg(total_count)
from rev_count '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет - 24.

<b>  Выводы</b>

Задачей проекта являлся анализ базы данных сервиса книг по подписке.

В базе данных — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.  

В ходе выполнения проекта выполнены следующие задания:  
- определено - сколько книг вышло после 1 января 2000 года - 819 книг из базы данных books удовлетворяют условию выпуска позднее 01.01.2000;  

- Для каждой книги произведен расчет количества обзоров и средней оценки;  

- Определено издательство, которое выпустило наибольшее число книг толще 50 страниц - Издательство Penguin Books	выпустило 42 книги толще 50 страниц - это максимальный показатель;  

- Определен автор с самой высокой средней оценкой книг для книг с 50 и более оценками - Самая высокая средняя оценка книг для книг с 50 и более оценками принадлежит J.K. Rowling/Mary GrandPré;

- Определено среднее количество обзоров от пользователей, которые поставили больше 48 оценок - согласно расчета это 24.